In [1]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split

In [2]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [3]:
# Load and preprocess the Fashion MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

# Split the training dataset into training and validation sets
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

batch_size = 256
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [4]:
for test in train_loader:
    print(len(test))
    break

2


In [4]:
# Define the LeNet model
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.fc1 = nn.Linear(16*4*4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, kernel_size=2, stride=2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, kernel_size=2, stride=2)
        x = x.view(-1, 16*4*4)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [6]:
# Initialize the model, loss function and optimizer
model = LeNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Validate the model
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {running_loss/len(train_loader)}, Validation Loss: {val_loss/len(val_loader)}, Validation Accuracy: {100 * correct / total}%')

Epoch 1/10, Training Loss: 0.6743754452864329, Validation Loss: 0.5185166169354256, Validation Accuracy: 80.675%
Epoch 2/10, Training Loss: 0.4341292844812075, Validation Loss: 0.38964214382019446, Validation Accuracy: 85.80833333333334%
Epoch 3/10, Training Loss: 0.37079134478171666, Validation Loss: 0.34774613245687586, Validation Accuracy: 87.19166666666666%
Epoch 4/10, Training Loss: 0.3344530634284019, Validation Loss: 0.3312272179792536, Validation Accuracy: 88.05%
Epoch 5/10, Training Loss: 0.3112851612965266, Validation Loss: 0.3159670872574157, Validation Accuracy: 88.53333333333333%
Epoch 6/10, Training Loss: 0.29443418857455256, Validation Loss: 0.3028417410368615, Validation Accuracy: 88.76666666666667%
Epoch 7/10, Training Loss: 0.27810966384410857, Validation Loss: 0.302999957603343, Validation Accuracy: 89.04166666666667%
Epoch 8/10, Training Loss: 0.26412793776392934, Validation Loss: 0.29328193713692907, Validation Accuracy: 89.56666666666666%
Epoch 9/10, Training Loss

In [7]:
# Test the model
model.eval()
test_loss = 0.0
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Test Loss: {test_loss/len(test_loader)}, Test Accuracy: {100 * correct / total}%')

Test Loss: 0.31758779135479287, Test Accuracy: 87.87%


In [4]:
torch.cuda

<module 'torch.cuda' from 'D:\\Miniconda\\envs\\AugustFirst\\lib\\site-packages\\torch\\cuda\\__init__.py'>